In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

## import dataset

In [2]:
dataset = pd.read_csv('../dataset/data01.csv')
dataset = dataset.iloc[:, 1:126]

# tX = dataset.iloc[:, 2:124]
# tY = dataset.iloc[:, 125]
# X = dataset.iloc[:, 2:123].values #idや名前は除去(特徴量ではない)
# y = dataset.iloc[:, 125].values #output(0 or 1) 

In [3]:
dataset.head()

,GAME,COUNTRY,YEAR,NAPFLT,GOBFLT,SERAMY,NWYAMY,GASAMY,AEGFLT,TUNAMY,...,MARFLT,ROMFLT,EDIAMY,EDIFLT,LONAMY,NAPAMY,NAOFLT,MUNAMY,LONFLT,OUTPUT
0,1,RUS,1901,[],[],[],[],[],[],[],...,[],[],[],[],[],[],[],[],[],4
1,1,AUS,1901,[],[],AUS,[],[],[],[],...,[],[],[],[],[],[],[],[],[],4
2,1,GER,1901,[],[],AUS,[],[],[],[],...,[],[],[],[],[],[],[],[],[],4
3,1,TUR,1901,[],[],AUS,[],[],[],[],...,[],[],[],[],[],[],[],[],[],4
4,1,FRA,1901,[],[],AUS,[],[],[],[],...,[],[],[],[],[],[],[],FRA,[],6


国別に分けて, outputのデータを最終年の補給地の数にする

In [4]:
rus_dataset = dataset[dataset.COUNTRY == 'RUS']
aus_dataset = dataset[dataset.COUNTRY == 'AUS']
ger_dataset = dataset[dataset.COUNTRY == 'GER']
tur_dataset = dataset[dataset.COUNTRY == 'TUR']
fra_dataset = dataset[dataset.COUNTRY == 'FRA']
eng_dataset = dataset[dataset.COUNTRY == 'ENG']
ita_dataset = dataset[dataset.COUNTRY == 'ITA']

In [5]:
eng_dataset.head()

,GAME,COUNTRY,YEAR,NAPFLT,GOBFLT,SERAMY,NWYAMY,GASAMY,AEGFLT,TUNAMY,...,MARFLT,ROMFLT,EDIAMY,EDIFLT,LONAMY,NAPAMY,NAOFLT,MUNAMY,LONFLT,OUTPUT
5,1,ENG,1901,[],[],AUS,[],[],[],[],...,[],[],[],[],[],[],[],FRA,[],3
12,1,ENG,1902,[],RUS,AUS,[],[],TUR,[],...,[],[],[],[],[],[],[],FRA,[],4
19,1,ENG,1903,[],RUS,AUS,[],[],TUR,[],...,[],[],[],[],[],[],[],FRA,[],4
26,1,ENG,1904,[],RUS,AUS,[],FRA,TUR,[],...,[],[],[],[],ENG,[],[],FRA,[],3
33,1,ENG,1905,FRA,RUS,AUS,[],FRA,TUR,[],...,[],[],[],[],ENG,[],[],FRA,[],3


In [6]:
from sklearn.utils import shuffle

In [7]:
tX = eng_dataset.iloc[:, 0:124]
tX = shuffle(tX)
tX = tX.drop_duplicates('GAME')
tX = tX.sort_values('GAME').iloc[:, 2:124]
tY = eng_dataset.drop_duplicates('GAME', keep = 'last').iloc[:,124]

In [8]:
tX.head()

,YEAR,NAPFLT,GOBFLT,SERAMY,NWYAMY,GASAMY,AEGFLT,TUNAMY,NWYFLT,GASFLT,...,GALAMY,MARFLT,ROMFLT,EDIAMY,EDIFLT,LONAMY,NAPAMY,NAOFLT,MUNAMY,LONFLT
26,1904,[],RUS,AUS,[],FRA,TUR,[],RUS,[],...,GER,[],[],[],[],ENG,[],[],FRA,[]
103,1908,ITA,RUS,AUS,[],FRA,AUS,[],ENG,[],...,RUS,[],[],[],ENG,ENG,[],ENG,GER,ENG
166,1902,ITA,RUS,AUS,RUS,FRA,TUR,[],ENG,[],...,RUS,[],ITA,ENG,ENG,ENG,[],ENG,GER,FRA
243,1902,ITA,RUS,AUS,RUS,RUS,TUR,[],ENG,[],...,RUS,ITA,ITA,ENG,ENG,ENG,ITA,ENG,GER,ENG
348,1901,ITA,RUS,AUS,RUS,FRA,TUR,[],ENG,ITA,...,RUS,ITA,ITA,ENG,ENG,ENG,ITA,FRA,ITA,ENG


In [9]:
tY.head()

47      3
152     4
229     5
341    10
460     4
Name: OUTPUT, dtype: int64

In [10]:
print(tX.shape,tY.shape)

(3093, 122) (3093,)


In [11]:
X = tX.values
y = tY.values

## Encoding categorical data
データの成形を行う

In [12]:
from sklearn.preprocessing import LabelEncoder, OneHotEncoder

##labelをエンコードする
le = LabelEncoder()
le.fit(['RUS','AUS','GER','TUR','FRA','ENG', 'ITA', '[]'])

for i in range(121):
    X[:, i+1] = le.transform(X[:, i+1])

### Gender
# labelencoder_X_2 = LabelEncoder()
# X[:, 2] = labelencoder_X_2.fit_transform(X[:, 2]) 

In [13]:
onehotencoder = OneHotEncoder(categorical_features = [1])
X = onehotencoder.fit_transform(X).toarray()

##  データセットと学習セットに分ける

In [14]:
# Splitting the dataset into the Training set and Test set
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

# Feature Scaling
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test) 

In [15]:
print(X_train.shape)

(2474, 128)


# モデルの作成

In [16]:
# Importing the Keras libraries and packages
import keras
from keras.models import Sequential
from keras.layers import Dense 

Using TensorFlow backend.


In [17]:
# Initialising the ANN
model = Sequential()

# Adding the input layer and the first hidden layer
model.add(Dense(units = 64, kernel_initializer = 'uniform', activation = 'relu', input_dim = 128))
# Adding the second hidden layer
model.add(Dense(units = 64, kernel_initializer = 'uniform', activation = 'relu'))
# Adding the output layer
model.add(Dense(units = 1, kernel_initializer = 'uniform', activation = 'relu'))

# Compiling the ANN
model.compile(optimizer = 'adam', loss = 'mean_squared_error', metrics = ['accuracy']) 

In [18]:
model.fit(X_train, y_train, batch_size = 10, epochs = 100) 

Epoch 1/100
2474/2474 [==============================] - 0s - loss: 17.5737 - acc: 0.1221          

In [ ]:
from sklearn.metrics import mean_squared_error
y_pred = model.predict(X_test)

# show its root mean square error
mse = mean_squared_error(y_test, y_pred)
print("KERAS REG RMSE : %.2f" % (mse ** 0.5))